<a href="https://colab.research.google.com/github/paskef/finetuned-model/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
from IPython import get_ipython
from IPython.display import display
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from google.colab import drive

In [ ]:
major_version, minor_version = torch.cuda.get_device_capability()

# Instalação de pacotes necessários
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes


### **Configurações e carregamento do modelo**

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Modelos de 4-bit disponíveis
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
]

# Carregar o modelo e o tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)


###  **Aplicação de PEFT (Parameter-Efficient Fine-Tuning)**

In [ ]:
model = FastLanguageModel.get_peft_model(
    model=model,
    r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)


### **Prompt e função de formatação**

In [ ]:
# Definição do prompt
prompt = """Abaixo está uma instrução que descreve uma situação, combinada com um input que fornece contexto adicional. Você deve responder como se fosse um profissional de saúde, caso a instrução não corresponda ao contexto de saúde, você não deve responder".

### Instrução:
{}

### Input:
{}

### Resposta:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = prompt.format(instruction, input, output) + EOS_TOKEN  # Adiciona o token de fim de sequência
        texts.append(text)
    return {"text": texts}


### *Carregando o Dataset*

In [ ]:
# Carregando o dataset
dataset = load_dataset("json", data_files="./data_cleaned.json", split="train")
# Formatação do dataset
dataset = dataset.map(formatting_prompts_func, batched=True)


### **Treinamento do Modelo**

In [ ]:

# Configurações do trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=4,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=14,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=30,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
    ),
)


Map (num_proc=4):   0%|          | 0/375 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

In [ ]:
drive.mount('/content/drive')

trainer.save_model("/content/drive/MyDrive/finetuning")
tokenizer.save_pretrained("/content/drive/MyDrive/finetuning")

Mounted at /content/drive


('/content/drive/MyDrive/finetuning/tokenizer_config.json',
 '/content/drive/MyDrive/finetuning/special_tokens_map.json',
 '/content/drive/MyDrive/finetuning/tokenizer.json')

### **Interação com o Chatbot**

In [ ]:
def run_chatbot(model, tokenizer, prompt):


  FastLanguageModel.for_inference(model)

  with open("respostas_chatbot.md", "w") as file:
    file.write("# Respostas do Chatbot\n\n")

    while True:
      user_input = input("Digite sua pergunta (ou 'sair' para encerrar): ")
      if user_input.lower() == "sair":
        print("Encerrando...")
        break

      try:
        # Tokeniza o input do usuário
        inputs = tokenizer(
            [
                prompt.format(
                    user_input,
                    "",
                    ""
                )
            ],
            return_tensors="pt"
        )

        # Gera a resposta
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            use_cache=True
        )

        decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        resposta = decoded_output.split("### Resposta:")[-1].strip()

        print(f"Chatbot: {resposta}\n")

        # Salva a interação no arquivo Markdown
        file.write(f"### Pergunta:\n{user_input}\n\n")
        file.write(f"### Resposta:\n{resposta}\n\n")

      except (RuntimeError, ValueError, TypeError) as e:
        print(f"Erro ao gerar a resposta: {e}")
        file.write(f"### Pergunta:\n{user_input}\n\n")
        file.write(f"### Resposta:\nErro ao gerar a resposta: {type(e).__name__}: {e}\n\n")

      except OSError as e:
        print(f"Erro ao salvar a resposta no arquivo: {e}")

      except Exception as e:
        print(f"Erro inesperado: {e}")

In [ ]:
run_chatbot(model, tokenizer, prompt)

Digite sua pergunta (ou 'sair' para encerrar): Estou com febre alta, o que faço?
Chatbot: Recomendo que beba bastante líquido para evitar desidratação, tome analgésicos como paracetamol ou ibuprofeno seguindo as instruções do fabricante, e procure um médico ou pronto-socorro para avaliação.

Digite sua pergunta (ou 'sair' para encerrar): Pode me ajudar com Programação Linear?
Chatbot: Desculpe, mas não posso te ajudar com Programação Linear. É um tópico complexo que requer conhecimento de matemática e programação. Recomendo procurar livros e cursos de especialização.

Digite sua pergunta (ou 'sair' para encerrar): Será que estou com alguma doença renal?
Chatbot: Não posso diagnosticar doenças.  Sintomas como inchaço nas pernas e pés, fadiga, urina espumosa, mudanças na quantidade de urina ou dor nas costas podem sugerir problemas renais.  Consulte um médico.

Digite sua pergunta (ou 'sair' para encerrar): sair
Encerrando...
